In [1]:
import numpy as np
import sympy as sp 
from sympy import assoc_legendre
import matplotlib.pyplot as plt

In [2]:
l,m = sp.symbols('l,m',real=True)
x,y,z = sp.symbols('x,y,z',real=True,positive=True)
r,theta,phi = sp.symbols('r,theta,phi')

# function to convert spherical coords to cartesian 
def SphericalToCartesian(f):
  return f.subs([(r,sp.sqrt(x**2 + y**2 + z**2)),
                 (theta,sp.acos(z/sp.sqrt(x**2 + y**2 + z**2))),
                 (phi,sp.asin(y/sp.sqrt(x**2 + y**2)))])

In [3]:
# function to generate basis function in cartisian coords 
def basisFunction(l,m):
  l+=1
  c_plus = sp.factorial(l-1) * (-2)**(sp.Abs(m)) /sp.factorial(l + m) * sp.cos(m * phi)
  c_negt = sp.factorial(l-1) * (-2)**(sp.Abs(m)) /sp.factorial(l + sp.Abs(m)) * sp.sin(sp.Abs(m) * phi)

  s_plus  = SphericalToCartesian(c_plus* r**l * assoc_legendre(l,m,sp.cos(theta)))
  s_minus = SphericalToCartesian(c_negt* r**l * assoc_legendre(l,sp.Abs(m),sp.cos(theta)))
  
  if m >= 0:
    return [sp.expand_trig(sp.diff(s_plus, axis)).simplify() for axis in [x,y,z]]
  else:
    return [sp.expand_trig(sp.diff(s_minus, axis)).simplify().expand() for axis in [x,y,z]]

In [4]:
# get the array of basis fucntion upto order l as a function of x,y,z
def basis_upto_order(l):
  Phi= [[basisFunction(i,j)[k] for i in range(l+1) for j in range(-i-1,i+2)] for k in range(3)]
  Phi_lambda = [sp.lambdify((x,y,z),Phi[i]) for i in range(3)]
  Phi_X,Phi_Y,Phi_Z = Phi_lambda[0],Phi_lambda[1],Phi_lambda[2]
  return Phi_X,Phi_Y,Phi_Z

In [5]:
# get the all the basis function upto order 3 
Phi_X,Phi_Y,Phi_Z = basis_upto_order(3)

In [6]:
# all basis fucntion at coordinate array, the order is Phi_z,Phi_X,Phi_Y
def all_basis_at_coordarray(coord_list):
  C_x = np.array(list(map(Phi_X,*coord_list.T)))
  C_y = np.array(list(map(Phi_Y,*coord_list.T)))
  C_z = np.array(list(map(Phi_Z,*coord_list.T)))
  C   = np.array([C_z,C_x,C_y])
  return C.T

In [7]:
# basis functions of oder l evaluated at coordinate array 
def basis_at_coordarray(coord_list,order):
  C_x = np.array(list(map(Phi_X,*coord_list.T)))
  C_y = np.array(list(map(Phi_Y,*coord_list.T)))
  C_z = np.array(list(map(Phi_Z,*coord_list.T)))
  C   = np.array([C_z,C_x,C_y])
  start_index, end_index =order*(order+2),(order +1)*(order + 3)
  return C.T[start_index:end_index]

In [8]:
# get value of all basis fucntion at a given point(probe location)
def all_basis_at_point(point,coord_list):
  return all_basis_at_coordarray(coord_list)[:,point,:]

In [9]:
# get the value of basis fucntion of given order at a probe location 
def basis_at_point(point,coord_list,order):
  return basis_at_coordarray(coord_list,order)[:,point,:]

In [10]:
# magnetometer positions 
pos_0 = np.array([[-1,0,0],[0,-1,0],[0,0,-1],[1,0,0],[0,1,0],[0,0,1]])     # along x,y,z axis 
pos_1 = np.array([[-1,-1,-1],[1,-1,-1],[1,1,-1],[-1,1,-1],
                  [-1,-1,1],[1,-1,1],[1,1,1],[-1,1,1]])                    # along the corners of a cube 
pos_2 = np.array([[-1,0,-1],[0,-1,-1],[1,0,-1],[0,1,-1],
                  [-1,-1,0],[1,-1,0],[1,1,0],[-1,1,0],
                  [-1,0,1],[0,-1,1],[1,0,1],[0,1,1]])                      # in between corners

In [11]:
# select 20 probe locations using (corners and inbetween the corners of cube)
pos_all = np.concatenate([pos_1,pos_2])  

In [12]:
# this gives at each basis function(along the rows) and at each probe location 
# what component is bigger(if z component is bigger ->0, x->1,y->2 )
index_array = np.argmax(np.abs(all_basis_at_coordarray(pos_all)),axis=2)

In [13]:
index_array=index_array[3:] # ignore the first three coeff(the constat fields)

In [14]:
index_array

array([[1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 2, 0, 2, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2],
       [1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2],
       [0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 2, 1, 0, 0, 0, 0, 2, 1, 2, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 2, 2, 2, 2, 2, 0, 2, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 0, 0, 0, 0, 1, 2, 1, 2],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 0, 0, 0, 0, 1, 2, 1, 2],
       [2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2],
       [0, 0, 0, 0, 0, 0, 0, 0, 2,

In [17]:
probe_location = [] # index of the coordinate array
coef_array     = [] # index of the coefficient array
probe_name     = [] # locate the probe and direction 

# first assign z harmonic coeff to coordinate(probe locations)
for i in range(len(index_array)):
  for j in range(len(index_array[i])):
    if i not in coef_array and j not in probe_location:
      if index_array[i][j] == 0:
        probe_location.append(j)
        coef_array.append(i)
        probe_name.append(f'coef {i:2d} assined to coord {j:2d} with z component')
      continue
# second assign x harmonic coeff to coordinate(probe locations)
for i in range(len(index_array)):
  for j in range(len(index_array[i])):
    if i not in coef_array and j not in probe_location:
      if index_array[i][j] == 1:
        probe_location.append(j)
        coef_array.append(i)
        probe_name.append(f'coef {i:2d} assined to coord {j:2d} with x component')
      continue
# third assign y harmonic coeff to coordinate(probe locations)
for i in range(len(index_array)):
  for j in range(len(index_array[i])):
    if i not in coef_array and j not in probe_location:
      if index_array[i][j] == 2:
        probe_location.append(j)
        coef_array.append(i)
        probe_name.append(f'coef {i:2d} assined to coord {j:2d} with y component')
      continue


In [18]:
probe_name

['coef  1 assined to coord  0 with z component',
 'coef  2 assined to coord  1 with z component',
 'coef  3 assined to coord  2 with z component',
 'coef  6 assined to coord  3 with z component',
 'coef  7 assined to coord  4 with z component',
 'coef  8 assined to coord 12 with z component',
 'coef  9 assined to coord  5 with z component',
 'coef 10 assined to coord 13 with z component',
 'coef 13 assined to coord 14 with z component',
 'coef 14 assined to coord  6 with z component',
 'coef 15 assined to coord  9 with z component',
 'coef 16 assined to coord  7 with z component',
 'coef 17 assined to coord  8 with z component',
 'coef 18 assined to coord 10 with z component',
 'coef 19 assined to coord 15 with z component',
 'coef  0 assined to coord 11 with x component',
 'coef  4 assined to coord 16 with x component',
 'coef 11 assined to coord 17 with x component',
 'coef 12 assined to coord 19 with x component',
 'coef 20 assined to coord 18 with x component']